In [ ]:
# ============================
# 📦 AI-Augmented Traffic Analyzer
# ============================

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest

# -----------------------------
# 1. Load and Prepare Data
# -----------------------------
df = pd.read_csv('traffic.csv')
print("Initial rows:")
print(df.head())

# Clean
df = df.dropna()

# Use only numeric features
features = ['tcp.srcport', 'tcp.dstport', 'frame.len']
X = df[features]

# -----------------------------
# 2. Anomaly Detection
# -----------------------------
iso = IsolationForest(contamination=0.05, random_state=42)
df['anomaly'] = iso.fit_predict(X)

# View anomalies
anomalies = df[df['anomaly'] == -1]
print("🚨 Total Anomalies Found:", len(anomalies))
print(anomalies.head())

# Plot anomalies in frame length
plt.figure(figsize=(12, 6))
plt.plot(df['frame.len'].values, label='Normal')
plt.scatter(anomalies.index, anomalies['frame.len'], color='red', label='Anomaly')
plt.title('Isolation Forest - Frame Length Anomalies')
plt.xlabel('Index')
plt.ylabel('Frame Length')
plt.legend()
plt.tight_layout()
plt.show()

# -----------------------------
# 3. Risk Scoring (Rule-Based)
# -----------------------------
def assign_risk(row):
    if row['tcp.dstport'] in [22, 23, 3389, 445, 80]:  # include 80 for demo
        return 'High'
    elif row['frame.len'] > 500:
        return 'Medium'
    else:
        return 'Low'

df['risk_level'] = df.apply(assign_risk, axis=1)

# Preview scoring
print("\nRisk Score Sample:")
print(df[['tcp.dstport', 'frame.len', 'anomaly', 'risk_level']].head())

# -----------------------------
# 4. Critical Alerts = Anomaly + Med/High Risk
# -----------------------------
critical_alerts = df[(df['anomaly'] == -1) & (df['risk_level'] != 'Low')]

print(f"\n⚠️ Critical Alerts Detected: {len(critical_alerts)}")
print(critical_alerts[['ip.src', 'ip.dst', 'tcp.dstport', 'frame.len', 'risk_level']].head())

# Visualize critical alert pattern by frame length
plt.figure(figsize=(10, 5))
plt.scatter(critical_alerts.index, critical_alerts['frame.len'], color='red', label='Critical Alert')
plt.title("Critical Alerts - Frame Length")
plt.xlabel("Index")
plt.ylabel("Frame Length")
plt.legend()
plt.tight_layout()
plt.show()

# -----------------------------
# 5. Risk Distribution Overview
# -----------------------------
sns.countplot(x='risk_level', data=df)
plt.title("Risk Level Distribution")
plt.xlabel("Risk Level")
plt.ylabel("Count")
plt.tight_layout()
plt.show()